# Predicting Thorax Diseases Using the ChestX-Ray14 Dataset and Convolutional Techniques

## Introduction

Dataset provided by the National Institute of Health at: https://nihcc.app.box.com/v/ChestXray-NIHCC

*Random subset provided [here](https://www.kaggle.com/nih-chest-xrays/sample)*

## Setup

In [43]:
import glob
import gzip
import os
import tarfile
import time
import warnings
from urllib.request import urlretrieve

import pandas as pd

import keras
from keras.applications import DenseNet121, ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten
from keras.metrics import AUC
from keras.models import load_model, Model
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import efficientnet.keras as efn

In [44]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [45]:
# Change to False to force local file system usage
USE_DRIVE = False

if USE_DRIVE:

    try:
        from google.colab import drive
        USE_DRIVE = True
        ROOT_DIR = './drive/My Drive'
    except:
        warnings.warn('Google Drive Not Found -- Using local file system')
        USE_DRIVE = False
        ROOT_DIR = '.'
else:
    ROOT_DIR = '.'

In [46]:
DATA_PATH = '/CXR14'
CHECKPOINT_PATH = '/models'

SAMPLE_RATE = 0.50
EPOCHS = 50
BATCH_SIZE = 32
CHECKPOINT_RATE = 2

CLASSES = [
  'Hernia',
  'Pneumonia',
  'Fibrosis',
  'Edema',
  'Emphysema',
  'Cardiomegaly',
  'Pleural_Thickening',
  'Consolidation',
  'Pneumothorax',
  'Mass',
  'Nodule',
  'Atelectasis',
  'Effusion',
  'Infiltration',
  'No Finding'
]

## Data Loading

In [48]:
if USE_DRIVE:
    drive.mount('/content/drive')

In [49]:
def batch_download_and_extract(path='.', first_n=None):

    # URLs for zip files containing ChestX-ray14 dataset from NIH
    links = [
        'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
        'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
        'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
        'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
        'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
        'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
        'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
        'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
        'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
        'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
        'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
        'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
    ]

    first_n = first_n or len(links)

    if first_n > len(links):
        raise('Number of files requested exceeds amount available')

    for idx, link in enumerate(links[:first_n]):
        fn = 'images_{:03d}.tar.gz'.format(idx+1)
        print('downloading', fn, '...')
        urlretrieve(link, fn)  # download the zip file

        tar = tarfile.open(fn, "r:gz")
        tar.extractall(path + '/images_{:03d}'.format(idx+1))
        tar.close()

        os.remove(fn)  # Remove remaining .tar file

    labels_url = 'https://nihcc.app.box.com/index.php?rm=box_download_shared_file&vanity_name=ChestXray-NIHCC&file_id=f_219760887468'
    urlretrieve(labels_url, path + '/Data_Entry_2017.csv')
  
    print("Download complete. Please check the checksums")

In [50]:
full_dir = "{0}{1}/full".format(ROOT_DIR, DATA_PATH)

if not os.path.isdir(full_dir):
    print('Data not present -- downloading now ...')
    os.makedirs(full_dir)
    batch_download_and_extract(full_dir)
else:
    print('Data directory already exists')

Data directory already exists


In [51]:
df = pd.read_csv("{}/Data_Entry_2017.csv".format(full_dir))
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [52]:
df['Finding Labels'] = df['Finding Labels'].apply(lambda s: s.split('|'))
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,[No Finding],0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,[Hernia],0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [53]:
# https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('Finding Labels')),
                          columns=mlb.classes_,
                          index=df.index))
df.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],...,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,0,1,58,M,PA,2682,2749,0.143,0.143,...,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,1,1,58,M,PA,2894,2729,0.143,0.143,...,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,2,1,58,M,PA,2500,2048,0.168,0.168,...,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,0,2,81,M,PA,2500,2048,0.171,0.171,...,0,0,0,0,0,1,0,0,0,0
4,00000003_000.png,0,3,81,F,PA,2582,2991,0.143,0.143,...,0,0,1,0,0,0,0,0,0,0


In [54]:
df = df[['Image Index'] + CLASSES]
df.head()

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration,No Finding
0,00000001_000.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,00000001_001.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,00000002_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,00000003_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
full_dir

'./CXR14/full'

In [56]:
img_paths =  glob.glob(full_dir + '/**/*.png', recursive=True)
img_paths[:5]

['./CXR14/full/images_001/images/00000626_000.png',
 './CXR14/full/images_001/images/00001075_014.png',
 './CXR14/full/images_001/images/00000682_003.png',
 './CXR14/full/images_001/images/00000273_008.png',
 './CXR14/full/images_001/images/00000870_005.png']

In [57]:
df['Image Index'] = df['Image Index'].apply(lambda x: next(p for p in img_paths if x in p))
df.head()

,Image Index,Hernia,Pneumonia,Fibrosis,Edema,Emphysema,Cardiomegaly,Pleural_Thickening,Consolidation,Pneumothorax,Mass,Nodule,Atelectasis,Effusion,Infiltration,No Finding
0,./CXR14/full/images_001/images/00000001_000.png,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,./CXR14/full/images_001/images/00000001_001.png,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
2,./CXR14/full/images_001/images/00000001_002.png,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
3,./CXR14/full/images_001/images/00000002_000.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,./CXR14/full/images_001/images/00000003_000.png,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
df = df.sample(frac=SAMPLE_RATE)

## Data Preparation

The ChestX-ray14 dataset is too large to fit entirely in memory when training; therefore, it's incrementally loaded via generator to reduce memory overhead. This is achieved using the Keras [Image Proprocessing](https://keras.io/preprocessing/image/) submodule.

In [59]:
# https://datascience.stackexchange.com/a/17445/91316

train_df, test_df = train_test_split(df, test_size=0.2)

print('Training/Validation Samples:  {}'.format(len(train_df)))
print('Test Samples:  {}'.format(len(test_df)))

Training/Validation Samples:  89696
Test Samples:  22424


In [60]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [61]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    subset='training',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=CLASSES,
    target_size=(224, 224)
)

Found 67272 validated image filenames.


In [62]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    subset='validation',
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 22424 validated image filenames.


In [63]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='Image Index',
    y_col=CLASSES,
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='raw',
    #classes=[],
    target_size=(224, 224)
)

Found 22424 validated image filenames.


## Modeling

In [64]:
full_dir + CHECKPOINT_PATH

'./CXR14/full/models'

In [65]:
if not os.path.exists(full_dir + CHECKPOINT_PATH):
    os.makedirs(full_dir + CHECKPOINT_PATH)

In [66]:
class TimeHistory(keras.callbacks.Callback):
    """Object used on keras callbacks to measure epoch training time

    Args:
        None

    Params:
        time (list): collection of times in seconds for each epoch's training

    """

    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

Three models will be implemented and their results compared:

1.   ResNet
2.   DenseNet
3.   EfficientNet


### ResNet

A pre-built ResNet model from the Keras library is used. Documentation on the model can be found [here](https://keras.io/applications/). Pre-trained weights from the ImageNet dataset are used.

In [67]:
resnet_base = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(resnet_base.output)

resnet = Model(input=resnet_base.input, outputs=output)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [68]:
resnet.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [69]:
resnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [70]:
resnet_time = TimeHistory()
resnet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
resnet_checkpoint = ModelCheckpoint(filepath=full_dir + CHECKPOINT_PATH + '/resnet-best.hdf5', 
                                    save_best_only=True)

resnet_history = resnet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[resnet_time, resnet_stopping, resnet_checkpoint]
)

Epoch 1/50
2103/2103 [==============================] - 1413s 672ms/step - loss: 0.1994 - binary_accuracy: 0.9297 - precision_2: 0.6383 - recall_2: 0.3129 - val_loss: 0.2624 - val_binary_accuracy: 0.9237 - val_precision_2: 0.6540 - val_recall_2: 0.3336
Epoch 2/50
2103/2103 [==============================] - 1381s 657ms/step - loss: 0.1901 - binary_accuracy: 0.9317 - precision_2: 0.6515 - recall_2: 0.3395 - val_loss: 0.2368 - val_binary_accuracy: 0.9248 - val_precision_2: 0.6637 - val_recall_2: 0.3277
Epoch 3/50
2103/2103 [==============================] - 1383s 658ms/step - loss: 0.1856 - binary_accuracy: 0.9326 - precision_2: 0.6708 - recall_2: 0.3219 - val_loss: 0.1707 - val_binary_accuracy: 0.9278 - val_precision_2: 0.6749 - val_recall_2: 0.3250
Epoch 4/50
 566/2103 [=======>......................] - ETA: 12:32 - loss: 0.1836 - binary_accuracy: 0.9321 - precision_2: 0.6751 - recall_2: 0.3245

KeyboardInterrupt: 

### DenseNet

In [37]:
densenet_base = DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(densenet_base.output)

densenet = Model(input=densenet_base.input, outputs=output)

29089792/29084464 [==============================] - 0s 0us/step


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [38]:
densenet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [41]:
densenet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [42]:
densenet_time = TimeHistory()
densenet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
densenet_checkpoint = ModelCheckpoint(filepath=full_dir + CHECKPOINT_PATH + '/densenet-best.hdf5', 
                                      save_best_only=True)

densenet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[densenet_time, densenet_stopping, densenet_checkpoint]
)

Epoch 1/2
1052/1052 [==============================] - 1445s 1s/step - loss: 0.1964 - binary_accuracy: 0.9296 - auc_2: 0.8785 - val_loss: 0.2293 - val_binary_accuracy: 0.9285 - val_auc_2: 0.8617
Epoch 2/2
1052/1052 [==============================] - 1377s 1s/step - loss: 0.1854 - binary_accuracy: 0.9322 - auc_2: 0.8963 - val_loss: 0.1697 - val_binary_accuracy: 0.9186 - val_auc_2: 0.8577


### EfficientNet

EfficientNet is a lightweight CNN architecture that is designed to require significantly less compute than other state of the art architectures on common transfer learning datasets.

Pre-built EfficientNet models built in Keras are used from the efficientnet library available on [GitHub](https://github.com/qubvel/efficientnet) and installable via PyPI.

In [33]:
efficientnet_base = efn.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
)
output = Dense(15, activation='sigmoid')(efficientnet_base.output)

efficientnet = Model(input=efficientnet_base.input, outputs=output)

71892992/71892840 [==============================] - 1s 0us/step


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [34]:
efficientnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 48) 1296        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 48) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [35]:
efficientnet.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['binary_accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

In [42]:
efficientnet_time = TimeHistory()
efficientnet_stopping = EarlyStopping(patience=5, restore_best_weights=True)
efficientnet_checkpoint = ModelCheckpoint(filepath=full_dir + CHECKPOINT_PATH + '/efficientnet-best.hdf5', 
                                          save_best_only=True)

efficientnet_history = efficientnet.fit_generator(
    generator=train_generator,
    epochs=EPOCHS,
    shuffle=True,
    validation_data=valid_generator,
    callbacks=[efficientnet_time, efficientnet_stopping, efficientnet_checkpoint]
)

Epoch 1/2
316/316 [==============================] - 211s 666ms/step - loss: 0.2144 - binary_accuracy: 0.9252 - precision_1: 0.5164 - recall_1: 0.3073 - val_loss: 0.2004 - val_binary_accuracy: 0.9292 - val_precision_1: 0.6148 - val_recall_1: 0.3223
Epoch 2/2
316/316 [==============================] - 203s 642ms/step - loss: 0.1935 - binary_accuracy: 0.9315 - precision_1: 0.6363 - recall_1: 0.3335 - val_loss: 0.2060 - val_binary_accuracy: 0.9310 - val_precision_1: 0.6489 - val_recall_1: 0.3385


## Results

### ResNet

In [ ]:
resnet = load_model(full_dir + CHECKPOINT_PATH + '/resnet-best.hdf5')

resnet_pred = resnet.predict_generator(
    generator=test_generator
)

In [ ]:
for idx, cls in enumerate(CLASSES):
    print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], resnet_pred[:,idx]))

### DenseNet

In [ ]:
densenet = load_model(full_dir + CHECKPOINT_PATH + '/densenet-best.hdf5')

densenet_pred = densenet.predict_generator(
    generator=test_generator
)

In [ ]:
for idx, cls in enumerate(CLASSES):
    print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], densenet_pred[:,idx]))

### EfficientNet

In [ ]:
efficientnet = load_model(full_dir + CHECKPOINT_PATH + '/efficientnet-best.hdf5')

efficientnet_pred = efficientnet.predict_generator(
    generator=test_generator
)

In [ ]:
for idx, cls in enumerate(CLASSES):
    print('{} AUC:  '.format(cls), roc_auc_score(test_df[cls], efficientnet_pred[:,idx]))